In [ ]:
# Install Microdf Package
!pip install git+https://github.com/PSLmodels/microdf.git

# Import Libraries
import numpy as np
import pandas as pd
import microdf as mdf
import plotly.express as px
import plotly.graph_objects as go

# Import Data - ASEC 2017-2019 to increase state level sample size
raw = pd.read_csv('https://github.com/ngpsu22/State_Child_Allowance_Income_Tax/raw/master/cps_00018.csv.gz')
fips = pd.read_csv('https://raw.githubusercontent.com/ngpsu22/disability_data/master/fips_state_codes.csv')
dc = {'code':'DC', 'state': 'District of Columbia'}
fips = fips.append(dc, ignore_index=True)

# Adjust column names
raw.columns = raw.columns.str.lower()
raw = raw.rename(columns={'statefip': 'state'})
del fips['Unnamed: 0']

# Change fip codes to state names
raw['state'] = raw['state'].astype(str)
raw['state'].replace({'1':'Alabama','2':'Alaska', '4': 'Arizona','5':'Arkansas',
                         '6': 'California', '8': 'Colorado', '9': 'Connecticut',
                         '10':'Delaware', '11': 'District of Columbia', '12':'Florida',
                         '13': 'Georgia','15':'Hawaii', '16':'Idaho','17':'Illinois',
                         '18':'Indiana', '19':'Iowa','20':'Kansas', '21': 'Kentucky',
                         '22':'Louisiana', '23': 'Maine', '24': 'Maryland',
                         '25':'Massachusetts', '26':'Michigan', '27': 'Minnesota',
                         '28':'Mississippi','29':'Missouri', '30': 'Montana',
                         '31': 'Nebraska', '32':'Nevada', '33': 'New Hampshire',
                         '34': 'New Jersey', '35': 'New Mexico', '36':'New York',
                         '37':'North Carolina', '38':'North Dakota', '39': 'Ohio',
                         '40':'Oklahoma', '41': 'Oregon', '42':'Pennsylvania',
                         '44':'Rhode Island','45':'South Carolina', '46':'South Dakota',
                         '47': 'Tennessee', '48':'Texas','49':'Utah','50':'Vermont',
                         '51':'Virginia', '53':'Washington', '54':'West Virginia',
                         '55':'Wisconsin', '56':'Wyoming'},inplace=True)

# Adjust to the most recent population estimates
TARGET_KIDS = 73_197_413  # As of July 1, 2019 per CBO.
kids_2017_2019 = raw[raw.age < 18].asecwt.sum()  
kids_scale_factor = TARGET_KIDS / kids_2017_2019
raw.asecwt *= kids_scale_factor

# Create demographic columns
raw['child'] = raw.age < 18
raw['adult'] = raw.age >= 18
raw['black'] = raw.race == 200
raw['white'] = raw.race == 100
raw['other'] = raw.race > 200

# Calculate the number of children and adults in each household
ages = raw.groupby(['spmfamunit', 'year'])[['child','adult']].sum()
ages.columns = ['total_children', 'total_adults']
raw = raw.merge(ages,left_on=['spmfamunit', 'year'], right_index=True)

# Adjust for NIUs
raw['statetax'].replace({9999999: 0}, inplace=True)
raw['taxinc'].replace({9999999: 0}, inplace=True)

# Create weighted columns
raw['weighted_child'] = raw.child * raw.asecwt
raw['weighted_state_tax'] = raw.stataxac * raw.asecwt
raw['weighted_taxinc'] = raw.taxinc * raw.asecwt

## NON-FUNDED ##

# Function for non-funded child allowance
def no_fund_pov(state, race, child_allowance, age_group):
  if state == 'US':
      target_persons = raw.copy(deep=True)    
  else:
      target_persons = raw[raw.state==state].copy(deep=True)

  target_persons['total_child_allowance'] = (
      target_persons.total_children * 12 * child_allowance)

  target_persons['new_spm_resources'] = (target_persons.spmtotres
                                         + target_persons.total_child_allowance)
  if age_group == 'child':
    target_persons = target_persons[target_persons.child]
  if age_group == 'adult':
    target_persons = target_persons[target_persons.adult] 
  
  if race == 'All':
    target_persons = target_persons
  if race == 'Black':
    target_persons = target_persons[target_persons.black]
  if race == 'White':
    target_persons = target_persons[target_persons.white]
  if race == 'Other':
    target_persons = target_persons[target_persons.other]
  
  target_persons['poor'] = (target_persons.new_spm_resources 
                            < target_persons.spmthresh)
  total_poor = (target_persons.poor * target_persons.asecwt).sum()
  target_pop = target_persons.asecwt.sum()

  return (total_poor / target_pop * 100).round(1)

def no_fund_pov_row(row):
  return no_fund_pov(row.state, row.race, row.child_allowance, row.age_group)

states = raw.state.unique().tolist()
summary = mdf.cartesian_product({'state': ['US'] + states,
                      'child_allowance': np.arange(0,501,50),
                       'age_group': ['child', 'adult','all'],
                       'race': ['Black','All', 'Other', 'White']})

# Merge fips codes
summary = (fips.merge(summary, on='state'))

# Apply the function across the data frame to calculate poverty rates.
summary['non_funded_poverty_rate'] = summary.apply(no_fund_pov_row, axis=1)

## FEDERALLY FUNDED

# Calculate the total amount of taxable income
total_taxinc = raw.weighted_taxinc.sum()

# Calculate the total number of children
child_pop = (raw.child * raw.asecwt).sum()

# Calculate federal tax rate per dollar of child allowance
fed_tax_rate_per_dollar_ca_monthly = (child_pop * 12) / total_taxinc

# Function for federally funded
def fed_pov(state, race, child_allowance, age_group):
  if state == 'US':
      target_persons = raw.copy(deep=True)    
  else:
      target_persons = raw[raw.state==state].copy(deep=True)
      
  target_persons['tax_increase'] = (
      fed_tax_rate_per_dollar_ca_monthly * child_allowance * target_persons.taxinc)
  target_persons['total_child_allowance'] = (
      target_persons.total_children * 12 * child_allowance)

  spmu = target_persons.groupby(['spmfamunit', 'year'])[['tax_increase']].sum()
  spmu.columns = ['total_tax_increase']
  target_persons = target_persons.merge(spmu,left_on=['spmfamunit', 'year'], right_index=True)
  target_persons['new_spm_resources'] = (target_persons.spmtotres
                                         + target_persons.total_child_allowance
                                         - target_persons.total_tax_increase)
  if age_group == 'child':
    target_persons = target_persons[target_persons.child]
  if age_group == 'adult':
    target_persons = target_persons[target_persons.adult] 
  
  if race == 'All':
    target_persons = target_persons
  if race == 'Black':
    target_persons = target_persons[target_persons.black]
  if race == 'White':
    target_persons = target_persons[target_persons.white]
  if race == 'Other':
    target_persons = target_persons[target_persons.other]
  
  target_persons['poor'] = (target_persons.new_spm_resources 
                            < target_persons.spmthresh)
  total_poor = (target_persons.poor * target_persons.asecwt).sum()
  target_pop = target_persons.asecwt.sum()

  return (total_poor / target_pop * 100).round(1)

def fed_pov_row(row):
  return fed_pov(row.state, row.race, row.child_allowance, row.age_group)

summary['fed_poverty_rate'] = summary.apply(fed_pov_row, axis=1)

## STATE FUNDED ##

# Calculate the total taxable income and total children in each state
state_groups_taxinc = raw.groupby(['state'])[['weighted_child','weighted_taxinc']].sum()
state_groups_taxinc.columns = ['state_children', 'state_taxable_income']
person2 = raw.merge(state_groups_taxinc, left_on=['state'], right_index=True)

# Calculate tax rate per dollar in each state
person2['state_rate_per_dollar_ca_mo'] = ((person2.state_children * 12) /
                                          (person2.state_taxable_income))

def state_pov(state, race, child_allowance, age_group):
  if state == 'US':
      target_persons = person2.copy(deep=True)    
  else:
      target_persons = person2[person2.state==state].copy(deep=True)
      
  target_persons['tax_increase'] = (
      person2.state_rate_per_dollar_ca_mo * child_allowance * target_persons.taxinc)
  target_persons['total_child_allowance'] = (
      target_persons.total_children * 12 * child_allowance)

  spmu = target_persons.groupby(['spmfamunit', 'year'])[['tax_increase']].sum()
  spmu.columns = ['total_tax_increase']
  target_persons = target_persons.merge(spmu,left_on=['spmfamunit', 'year'], right_index=True)
  target_persons['new_spm_resources'] = (target_persons.spmtotres
                                         + target_persons.total_child_allowance
                                         - target_persons.total_tax_increase)
  if age_group == 'child':
    target_persons = target_persons[target_persons.child]
  if age_group == 'adult':
    target_persons = target_persons[target_persons.adult] 
  
  if race == 'All':
    target_persons = target_persons
  if race == 'Black':
    target_persons = target_persons[target_persons.black]
  if race == 'White':
    target_persons = target_persons[target_persons.white]
  if race == 'Other':
    target_persons = target_persons[target_persons.other]
  
  target_persons['poor'] = (target_persons.new_spm_resources 
                            < target_persons.spmthresh)
  total_poor = (target_persons.poor * target_persons.asecwt).sum()
  target_pop = target_persons.asecwt.sum()

  return (total_poor / target_pop * 100).round(1)


def state_pov_row(row):
  return state_pov(row.state, row.race, row.child_allowance, row.age_group)

summary['state_poverty_rate'] = summary.apply(state_pov_row, axis=1)

In [ ]:
person2['total_people'] = person2.total_adults + person2.total_children
person2['original_resources_per_person'] = (person2.spmtotres /
                                            person2.total_people)

def distribution(state, child_allowance):
  if state == 'US':
      target_persons = person2.copy(deep=True)    
  else:
      target_persons = person2[person2.state==state].copy(deep=True)

  target_persons['tax_increase'] = (
      person2.state_rate_per_dollar_ca_mo * child_allowance *  target_persons.taxinc)
  target_persons['total_child_allowance'] = (
      target_persons.total_children * 12 * child_allowance)

  spmu = target_persons.groupby(['spmfamunit', 'year'])[['tax_increase']].sum()
  spmu.columns = ['total_tax_increase']
  target_persons = target_persons.merge(spmu,left_on=['spmfamunit', 'year'], right_index=True)
  target_persons['new_spm_resources'] = (target_persons.spmtotres
                                         + target_persons.total_child_allowance
                                         - target_persons.total_tax_increase)
  
  target_persons['new_resources_per_person'] = target_persons.new_spm_resources / target_persons.total_people

  deciles = mdf.quantile_chg(target_persons, target_persons, 'original_resources_per_person', 
                            'new_resources_per_person', 'asecwt', 'asecwt',
                            np.arange(0.1,1.0,0.1))
  deciles = deciles.transpose()
  deciles.columns = ['old', 'new']
  deciles['percent_change'] = (((deciles.new - deciles.old) /
                                        deciles.old) * 100).round(1)
  deciles['deciles'] = ['10th', '20th','30th', '40th','50th',
                     '60th','70th', '80th', '90th']

  deciles['state'] = state
  deciles['monthly_child_allowance'] = child_allowance
  return deciles

state_distribution = mdf.cartesian_product({'state': ['US'] + states,
                            'child_allowance': np.arange(0, 501, 50)})

l = []
for index, row in state_distribution.iterrows():
  l.append(distribution(row.state, row.child_allowance))

deciles = pd.concat(l)

deciles["color"] = np.where(deciles["percent_change"] >= 0, 'green', "red")